# Loan predictions

## Problem Statement

We want to automate the loan eligibility process based on customer details that are provided as online application forms are being filled. You can find the dataset [here](https://drive.google.com/file/d/1h_jl9xqqqHflI5PsuiQd_soNYxzFfjKw/view?usp=sharing). These details concern the customer's Gender, Marital Status, Education, Number of Dependents, Income, Loan Amount, Credit History and other things as well. 

|Variable| Description|
|: ------------- |:-------------|
|Loan_ID| Unique Loan ID|
|Gender| Male/ Female|
|Married| Applicant married (Y/N)|
|Dependents| Number of dependents|
|Education| Applicant Education (Graduate/ Under Graduate)|
|Self_Employed| Self employed (Y/N)|
|ApplicantIncome| Applicant income|
|CoapplicantIncome| Coapplicant income|
|LoanAmount| Loan amount in thousands|
|Loan_Amount_Term| Term of loan in months|
|Credit_History| credit history meets guidelines|
|Property_Area| Urban/ Semi Urban/ Rural|
|Loan_Status| Loan approved (Y/N)



### Explore the problem in following stages:

1. Hypothesis Generation – understanding the problem better by brainstorming possible factors that can impact the outcome
2. Data Exploration – looking at categorical and continuous feature summaries and making inferences about the data.
3. Data Cleaning – imputing missing values in the data and checking for outliers
4. Feature Engineering – modifying existing variables and creating new ones for analysis
5. Model Building – making predictive models on the data

## 1. Hypothesis Generation

Generating a hypothesis is a major step in the process of analyzing data. This involves understanding the problem and formulating a meaningful hypothesis about what could potentially have a good impact on the outcome. This is done BEFORE looking at the data, and we end up creating a laundry list of the different analyses which we can potentially perform if data is available.

#### Possible hypotheses
Which applicants are more likely to get a loan

1. Applicants having a credit history 
2. Applicants with higher applicant and co-applicant incomes
3. Applicants with higher education level
4. Properties in urban areas with high growth perspectives

Do more brainstorming and create some hypotheses of your own. Remember that the data might not be sufficient to test all of these, but forming these enables a better understanding of the problem.

## 2. Data Exploration
Let's do some basic data exploration here and come up with some inferences about the data. Go ahead and try to figure out some irregularities and address them in the next section. 

In [226]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [227]:

df = pd.read_csv("data.csv") 
dfs = df.copy()
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


One of the key challenges in any data set are missing values. Lets start by checking which columns contain missing values.

In [228]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [229]:
nulldf = df[df.isna().any(axis=1)]
nulldf.shape

(134, 13)

In [230]:
df.loc[(df['CoapplicantIncome'] > 0) & (df['Married'] == 'Yes') & (df['Gender'] == 'Male')].shape
# not married and co 0       143 77m 76f
# married and co 0           128 113m 15f
# not married and co > 0      70 53m 17f
# married and co > 0         270 244m 26f

(244, 13)

In [272]:
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
11,LP001027,Male,Yes,2,Graduate,NaN,2500,1840.0,109.0,360.0,1.0,Urban,Y
16,LP001034,Male,No,1,Not Graduate,No,3596,0.0,100.0,240.0,NaN,Urban,Y
19,LP001041,Male,Yes,0,Graduate,NaN,2600,3500.0,115.0,NaN,1.0,Urban,Y
23,LP001050,NaN,Yes,2,Not Graduate,No,3365,1917.0,112.0,360.0,0.0,Rural,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,LP002933,NaN,No,3+,Graduate,Yes,9357,0.0,292.0,360.0,1.0,Semiurban,Y
597,LP002943,Male,No,NaN,Graduate,No,2987,0.0,88.0,360.0,0.0,Semiurban,N
600,LP002949,Female,No,3+,Graduate,NaN,416,41667.0,350.0,180.0,NaN,Urban,N
601,LP002950,Male,Yes,0,Not Graduate,NaN,2894,2792.0,155.0,360.0,1.0,Rural,Y


In [231]:
dff=df.dropna()
dff.shape

(480, 13)

Look at some basic statistics for numerical variables.

In [232]:
df['Gender'].value_counts()

Male      489
Female    112
Name: Gender, dtype: int64

In [233]:
df['Married'].value_counts()

Yes    398
No     213
Name: Married, dtype: int64

In [234]:
df['Dependents'].value_counts()

0     345
1     102
2     101
3+     51
Name: Dependents, dtype: int64

In [235]:
df['Education'].value_counts()

Graduate        480
Not Graduate    134
Name: Education, dtype: int64

In [236]:
df['Self_Employed'].value_counts()

No     500
Yes     82
Name: Self_Employed, dtype: int64

In [237]:
df['ApplicantIncome'].describe()

count      614.000000
mean      5403.459283
std       6109.041673
min        150.000000
25%       2877.500000
50%       3812.500000
75%       5795.000000
max      81000.000000
Name: ApplicantIncome, dtype: float64

In [238]:
bins = [0, 2878,3813,5796,np.inf]
names= [0,1,2,3]
df['ApplicantIncomeBin']=pd.cut(df['ApplicantIncome'],bins,labels=names)

In [239]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,ApplicantIncomeBin
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y,3
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,2
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,0
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,3


In [240]:
df['CoapplicantIncome'].describe()

count      614.000000
mean      1621.245798
std       2926.248369
min          0.000000
25%          0.000000
50%       1188.500000
75%       2297.250000
max      41667.000000
Name: CoapplicantIncome, dtype: float64

In [241]:
bins = [0,10,1189,2298,np.inf]
names= [0,1,2,3]
df['CoapplicantIncomeBin']=pd.cut(df['CoapplicantIncome'],bins,labels=names)

In [242]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,ApplicantIncomeBin,CoapplicantIncomeBin
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y,3,NaN
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,2,2
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1,NaN
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,0,3
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,3,NaN


In [243]:
df['LoanAmount'].describe()

count    592.000000
mean     146.412162
std       85.587325
min        9.000000
25%      100.000000
50%      128.000000
75%      168.000000
max      700.000000
Name: LoanAmount, dtype: float64

In [244]:
bins = [0, 101,129,169,np.inf]
names= [0,1,2,3]
df['LoanAmountBin']=pd.cut(df['LoanAmount'],bins,labels=names)

In [245]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,ApplicantIncomeBin,CoapplicantIncomeBin,LoanAmountBin
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y,3,NaN,NaN
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N,2,2,1
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y,1,NaN,0
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y,0,3,1
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y,3,NaN,2


In [246]:
df['Loan_Amount_Term'].value_counts()

360.0    512
180.0     44
480.0     15
300.0     13
84.0       4
240.0      4
120.0      3
36.0       2
60.0       2
12.0       1
Name: Loan_Amount_Term, dtype: int64

In [247]:
df['Credit_History'].value_counts()

1.0    475
0.0     89
Name: Credit_History, dtype: int64

In [248]:
df['Property_Area'].value_counts()

Semiurban    233
Urban        202
Rural        179
Name: Property_Area, dtype: int64

In [249]:
df['Loan_Status'].value_counts()

Y    422
N    192
Name: Loan_Status, dtype: int64

In [250]:
df.to_csv('df.csv')

In [273]:
df.isnull().sum()

Loan_ID                   0
Gender                   13
Married                   3
Dependents               15
Education                 0
Self_Employed            32
ApplicantIncome           0
CoapplicantIncome         0
LoanAmount               22
Loan_Amount_Term         14
Credit_History           50
Property_Area             0
Loan_Status               0
ApplicantIncomeBin        0
CoapplicantIncomeBin    273
LoanAmountBin            22
dtype: int64

## 3. Data Cleaning

This step typically involves imputing missing values and treating outliers. 

In [251]:
dff.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [252]:
dfs['Gender'] = LabelEncoder().fit_transform(dfs['Gender'])

In [253]:
dfs['Married'] = LabelEncoder().fit_transform(dfs['Married'])

In [254]:
dfs['Education'] = LabelEncoder().fit_transform(dfs['Education'])

In [255]:
dfs['Self_Employed'] = LabelEncoder().fit_transform(dfs['Self_Employed'])

In [256]:
dfs['Loan_Amount_Term'] = LabelEncoder().fit_transform(dfs['Loan_Amount_Term'])

In [257]:
dfs['Property_Area'] = LabelEncoder().fit_transform(dfs['Property_Area'])

In [258]:
dfs['Loan_Status'] = LabelEncoder().fit_transform(dfs['Loan_Status'])

In [259]:
dfs['Dependents'] = LabelEncoder().fit_transform(dfs['Dependents'])

In [260]:
dfs = dfs.drop(['Loan_ID'],axis=1)

In [261]:
dfs.isnull().sum()

Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term      0
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [262]:
dfnn = dfs.loc[(dfs['LoanAmount'].isna()) | (dfs['Credit_History'].isna())].reset_index()

In [263]:
dfn = dfs.loc[(dfs['LoanAmount'].isna()) | (dfs['Credit_History'].isna())].reset_index()

In [264]:
dfn.drop(dfn[(dfn['LoanAmount'].isna()) & (dfn['Credit_History'].isna())].index,inplace=True)

In [274]:
dfnch = dfn.loc[dfn['Credit_History'].isna()]

In [275]:
dfnch.head()

,index,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,16,1,0,1,1,0,3596,0.0,100.0,6,NaN,2,1
2,24,1,1,1,0,2,3717,2925.0,151.0,8,NaN,1,0
3,30,1,1,1,0,2,4166,3369.0,201.0,8,NaN,2,0
5,42,1,1,0,0,0,2400,0.0,75.0,8,NaN,2,1
7,79,1,1,3,1,1,3333,2166.0,130.0,8,NaN,1,1


In [267]:
dfsf=dfs.dropna()

### Setting variables to predict NaN

In [268]:
X_train= dfsf.drop(['Credit_History'],axis=1)

In [269]:
y_train=dfsf['Credit_History']

In [270]:
X_test=dfnch.drop(['Credit_History'],axis=1)

In [271]:
y_test=dfnch['Credit_History']

### KNN

In [180]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3, metric = 'minkowski', p = 2)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [181]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred_knn = knn.predict(X_test)

In [182]:
dft = pd.DataFrame(y_pred_knn, y_test)

### Logistic Regression

In [183]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state = 0)
logreg.fit(X_train, y_train)

/Users/scm/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [185]:
y_pred_logreg = logreg.predict(X_test)

In [186]:
dflogreg = pd.DataFrame(y_pred_logreg,y_test)

In [187]:
dflogreg

,0
Credit_History,
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0
1.0,1.0


In [188]:
dfnch['ch_pred']=y_pred_knn

<ipython-input-188-95970773b6c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfnch['ch_pred']=y_pred_knn


In [189]:
dfnch['Credit_History']=dfnch['ch_pred']
dfnch.drop('ch_pred',axis=1,inplace=True)

<ipython-input-189-9967fa842d39>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfnch['Credit_History']=dfnch['ch_pred']
/Users/scm/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [190]:
dfnch

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,1,0,1,1,0,3596,0.0,100.0,6,1.0,2,1
2,1,1,1,0,2,3717,2925.0,151.0,8,1.0,1,0
3,1,1,1,0,2,4166,3369.0,201.0,8,1.0,2,0
5,1,1,0,0,0,2400,0.0,75.0,8,1.0,2,1
7,1,1,3,1,1,3333,2166.0,130.0,8,1.0,1,1
9,1,1,0,0,0,6000,2250.0,265.0,8,1.0,1,0
10,1,1,2,1,0,3333,2000.0,99.0,8,1.0,1,1
15,1,1,1,0,0,2214,1398.0,85.0,8,1.0,2,1
16,0,0,0,0,0,3692,0.0,93.0,8,1.0,0,1
18,1,1,0,0,0,6080,2569.0,182.0,8,1.0,0,0


In [202]:
df_cred_hist_full = pd.concat([dfnch, dfn])

In [203]:
df_cred_hist_full.dropna(subset= ['Credit_History'], inplace=True)

In [205]:
df_cred_hist_full.dropna(subset=['LoanAmount'],inplace=True)


In [206]:
df_cred_hist_full


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,1,0,1,1,0,3596,0.0,100.0,6,1.0,2,1
2,1,1,1,0,2,3717,2925.0,151.0,8,1.0,1,0
3,1,1,1,0,2,4166,3369.0,201.0,8,1.0,2,0
5,1,1,0,0,0,2400,0.0,75.0,8,1.0,2,1
7,1,1,3,1,1,3333,2166.0,130.0,8,1.0,1,1
9,1,1,0,0,0,6000,2250.0,265.0,8,1.0,1,0
10,1,1,2,1,0,3333,2000.0,99.0,8,1.0,1,1
15,1,1,1,0,0,2214,1398.0,85.0,8,1.0,2,1
16,0,0,0,0,0,3692,0.0,93.0,8,1.0,0,1
18,1,1,0,0,0,6080,2569.0,182.0,8,1.0,0,0


### Imputing Missing Values

Missing values may not always be NaNs. For instance, the `Loan_Amount_Term` might be 0, which does not make sense.



Impute missing values for all columns. Use the values which you find most meaningful (mean, mode, median, zero.... maybe different mean values for different groups)

In [73]:
from sklearn.preprocessing import LabelEncoder
#dff['Married'] = LabelEncoder().fit_transform(dff['Married'])

### Extreme values
Try a log transformation to get rid of the extreme values in `LoanAmount`. Plot the histogram before and after the transformation

Combine both incomes as total income and take a log transformation of the same.

## 4. Building a Predictive Model

Try paramater grid search to improve the results

## 5. Using Pipeline
If you didn't use pipelines before, transform your data prep, feat. engineering and modeling steps into Pipeline. It will be helpful for deployment.

The goal here is to create the pipeline that will take one row of our dataset and predict the probability of being granted a loan.

`pipeline.predict(x)`

In [16]:
dff.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [23]:
binary_features = ['Gender','Married','Education','Self_Employed','Credit_History','Loan_Status']
categorical_features = ['Dependents','Loan_Amount_Term','Property_Area']
numerical_features = ['ApplicantIncome', 'CoapplicantIncome','LoanAmount']
binary_transformer = Pipeline(steps=[
    ('imput',le)
])
preprocessor = ColumnTransformer(
    transformers = [
        ('bin',binary_transformer,binary_features)
    ])

my_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

In [26]:
my_pipeline.fit(dfs)

ValueError: y should be a 1d array, got an array of shape () instead.

In [18]:
dfs['Married'] = LabelEncoder().fit_transform(dfs['Married'])

## 6. Deploy your model to cloud and test it with PostMan, BASH or Python

In [20]:
dfs['Married'].value_counts()

1    398
0    213
2      3
Name: Married, dtype: int64